This notebook include program code for an automated assignment algorithm. It has been used several times for university courses to opimize the distribution of students according to their course preferences.

## Automated Course Assignments

The following program enables automated assignment of course participants according to their preferences. You can only input *2 to 10 * preferences. After reading in the necessary data a genetig algorithm will be used. The code for this algorithm is written by **Shannon and Mckinney** and used in the article <a href="http://www.aaai.org/ocs/index.php/FLAIRS/FLAIRS11/paper/download/2596/3067" target="_blank">An Evolutionary Algorithm for Assigning Students to Courses</a>. Here, parts of the program code are adapted for additional input possibilites.    

Keep in mind that this program is based upon a heuristic approach and it might not achieve the optimal solution. Depending on the preferences there might not even exist one single solution. In case you are unhappy with the program assignment, you can run the program again, which can lead to slightly different outcomes. Please note that the input language as well as the code documentation are written in German. When I have the time I will set up an English translation.

In order for the program to run properly, the input file containing the course participant data has to match the following scheme exactly:

        ID   name surname   preference1   preference2   preference3
        ID   name surname   preference1   preference2   preference3
        ..
        
The columns are separated by tabulators. This is especially important for the preference information as these will be used by the program to optimize assignments.    


I also compiled the whole program to a windows standalone ```.exe``` which can be downloaded [here](https://www.dropbox.com/s/rlbtvcmf80vsumb/Kurseinteilung.zip?dl=1).

## Program Code

In [7]:
import random
import sys
import time


def switch(dataFile,maxCls, studPref):
    # ################################################# #
    # Erstellung der students.txt Datei.                #
    # # # # # # # # # # # # # # # # # # # # # # # # # # #
    # dataFile - Datei mit Rohdaten                     #
    # maxCls - maximale Anzahl der Studierenden in      #
    #          einem Kurs                               # 
    # studPref - Anzahl an Praeferenzen                 #
    # ################################################# #
    """Erstellung der students.txt Datei."""

    # Rohdaten einlesen
    try:
        f = open(dataFile,'r')
        fcontents = f.readlines()
        f.close()
    except IOError:
        print "Die Datei mit den Studierendeninformationen wurde nicht gefunden!", dataFile + "!"
        sys.exit(1)

    # students.txt oeffnen
    try:
        s = open("students.txt",'w')
    except IOError:
        print "Die students.txt Datei konnte nicht erstellt werden!"
        sys.exit(1)

    # Klasseninformationen
    classes = dict()

    # Einteilungsinformationen
    for i in fcontents:
        data = i.split()
        for j in data[-studPref:]:
              # Hier muessen je nach Anzahl der Kurse Anpassungen durchgefuehrt werden
            if j in classes:
                classes[j].append(data[0])
            else:
                classes[j] = [data[0]]

    print classes

    # Moegliche Problemkurse identifizieren
    print "\tUeberpruefe Kurse die nicht gefuellt werden koennen..."
    for i in classes:
	print i
        if len(classes[i]) < maxCls:
            print "\t\Kurs", i, "kann nicht komplett gefuellt werden."
    print "\tUeberpruefung abgeschlossen."

    # students.txt erstellen
    classList = ""
    for i in classes:
        classList += str(i) + " "
    classList += "\n"
    s.write(classList)
    
    for i in fcontents:
        data = i.split()
        writeMe = str(data[0])
        for j in data[-studPref:]:
              # Hier muessen je nach Anzahl der Kurse Anpassungen durchgefuehrt werden
            writeMe += "\t" + str(j)
        writeMe += "\n"
        s.write(writeMe)

    # students.txt schliessen
    s.close()




# Einteilung durch den Algorithmus


class Ordering:
    def __init__(self,numStudent,maxCls,trial,dataFile,studPref):
        # ######################################## #
        # Initialisiert einen neuen Ordnungsalgorithmus #
        # ######################################## #

            #########################
            #####   Konstanten   #####
            
        # Einteilung
        self.NUMSTD = numStudent    #$ Anzahl der Studierenden
        self.MAXCLSSIZE = maxCls    # Maximale Anzahl an Studierenden in einer Klasse
        self.DATAFILE = dataFile
	self.studPref = studPref

        # Algorithmus
        self.POP = 200
        self.C = 1  # prozentualer Anzeil der Crossover Zeit
        self.M = .01   # prozentualer Anzeil der zu mutierenden Bits in der Population
        self.RUNS = 1000 # Anzahl der Generierungen
           
            #####   Konstanten   #####
            #########################

            #####################
            #####   Dateien   #####
        try:
            # Einteilungsinformationen
            f = open("students.txt",'r')
            self.infoList = self.getInfoLists(f)
            f.close()

            # Output der Analyse
            #fa = open("order" + str(trial) + ".txt",'w')
        except IOError:
            sys.exit(1)
            #####   Dateien   #####
            #####################

        # prepopulation erstellen
        self.pdna = self.createPrePop()

        # initialpopulation erstellen
        self.dna = self.choosePop()

        # prepopulation fuer spaeter leeren
        self.pdna = dict()

        # maximale fitness und liste mit zugehoerigen chromosomen extrahieren
        fx, fxs = self.getMaxFitChros()

        # Daten zusammenfuegen
        #fa.write("Population size "+str(self.POP)+ " generations "+ str(self.RUNS)+"\n")
        #fa.write("0 " + str(fx) + " " + str(len(self.dna[str(fx)])) + "\n")

        # Start Generationen
        for i in range(self.RUNS):
            # notify of start
            #print("Starting Generation " + str(i+1) + "...")

            # Liste der Fitness Ratios
            fRatios = self.getFRatios()

            # generieren bis die Anzahl der Chromosomen = POP
            count = 0
            while count < self.POP:
                count += self.generate(fRatios)

            # die generierte Population mutieren
            self.mutate()

            # Parent und Children Populationen zusammenfuegen
            self.merge()

            # Vertauschen
            self.pdna = self.dna

            # die obere Haelfte nach der Fitness waehlen
            self.dna = self.choosePop()
            self.pdna = dict()

            # maximale fitness und liste mit zugehoerigen chromosomen extrahieren
            fx, fxs = self.getMaxFitChros()

            # Daten zusamenfuegen
            #fa.write(str(i+1) + " " + str(fx) + " " +
            #         str(len(self.dna[str(fx)])) + "\n")

            # 
            #print("Generation " + str(i+1) + studPref" Complete.")

        # Ergebnisse erzeugen
        self.createOutput(fxs, self.DATAFILE)
        
        # Dateien schliessen
        
    # # # # # # # # # # # # # # # # # # # # # # #
    # Ende der Initialisierung  #
    # # # # # # # # # # # # # # # # # # # # # # #


    def createPrePop(self):
        # ###########################   #
        # Rueckgabe der Pre-Population. #
        # ###########################   #
        """Rueckgabe der Pre-Population."""

        
	

	# Diktionaer erstellen
        pop = dict()

        # Population substantiieren
        for i in range(self.POP * 2):
            # Chromosom
            chro = range(self.NUMSTD)
            random.shuffle(chro)

            # Fitness
            f = self.getFitness(chro,self.studPref)

            # zur Population hinzufuegen
            if str(f) in pop.keys():
                pop[str(f)].append(chro)
            else:
                pop[str(f)] = [chro]

        # Rueckgabe der Pre-Population
        return pop

    


    def getFitness(self,chromosome, studPref, detail = False):
        # ##################################################### #
        # Rueckgabe der Chromosom Fitness                       #
        # Substantierung self.classes.                          #
        # Alle nicht zugewiesenen Studierenden in               #
        # self.classes["unassigned"]                            #
        #   * self.classes wird fuer Verbesserungen verwendet.  #
        # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
        # chromosome - Zielchromosom                            #
        # detail - Satisfaction Counter                         #
        # ##################################################### #
        """Rueckgabe der Chromosome Fitness."""

        # Counter           
        # Generierung der Variablen anhand der Anzahl der Praeferenzen aus studPref
        
	self.studPref = studPref


        dicPref = {1: "numFirst", 2: "numSecond", 3: "numThird", 4: "numFourth", 5: "numFifth", 6: "numSixth", \
                       7: "numSeventh", 8: "numEight", 9: "numNinth", 10: "numTenth"}

        Pref = {}
        for x in range(1,studPref+1):
        	Pref[x] = dicPref[x]

        for key, value in Pref.iteritems():
            exec("%s = %s" % (value, 0))
	   
            

        numUn = 0

        # Kurse
        self.classes = dict()
        for i in self.infoList[0]:
            self.classes[str(i)] = list()
        self.classes["unassigned"] = list()

        # Studentenaufteilung
        for i in chromosome:
            # i ist der jeweilige Student

            # Zuweisung
            assigned = False
            for j in range(1,len(self.infoList[i+1])):
                cls = str(self.infoList[i+1][j])
                if len(self.classes[cls]) < self.MAXCLSSIZE:
                    self.classes[cls].append(i+1)
                    assigned = True
                    which = j
                    break

            # Auswahl
            if assigned:
                if which == 1:
                    numFirst += 1
                elif which == 2:
                    numSecond += 1
                elif which == 3:
                    numThird += 1
                elif which == 4:
                    numFourth += 1
                elif which == 5:
                    numFifth += 1
                elif which == 6:
                    numSixth += 1
                elif which == 7:
                    numSeventh += 1
                elif which == 8:
                    numEigth += 1
                elif which == 9:
                    numNinth += 1
                elif which == 10:
                    numTenth += 1
            else:
                numUn += 1
                self.classes["unassigned"].append(i+1)

        # Fitness berechnen
        if studPref ==2:
            satisfaction =  2*numFirst + 1*numSecond
        if studPref ==3:
            satisfaction =  3*numFirst + 2*numSecond + 1*numThird
        if studPref ==4:
            satisfaction =  4*numFirst + 3*numSecond + 2*numThird + 1*numFourth
        if studPref ==5:
            satisfaction =  5*numFirst + 4*numSecond + 3*numThird + 2*numFourth + 1*numFifth
        if studPref ==6:
            satisfaction =  6*numFirst + 5*numSecond + 4*numThird + 3*numFourth + 2*numFifth + 1*numSixth
        if studPref ==7:
            satisfaction =  7*numFirst + 6*numSecond + 5*numThird + 4*numFourth + 3*numFifth + 2*numSixth + 1*numSeventh
        if studPref ==8:
            satisfaction =  8*numFirst + 7*numSecond + 6*numThird + 5*numFourth + 4*numFifth + 3*numSixth + 2*numSeventh \
            + 1* NumEigth
        if studPref ==9:
            satisfaction =  9*numFirst + 8*numSecond + 7*numThird + 6*numFourth + 5*numFifth + 4*numSixth + 3*numSeventh \
            + 2*NumEigth + 1*NumNinth
        if studPref ==10:
            satisfaction =  10*numFirst + 9*numSecond + 8*numThird + 7*numFourth + 6*numFifth + 5*numSixth + 4*numSeventh \
            + 3*numEigth + 2*numNinth + 1*numTenth
        
        

        
        # Hier muessen je nach Anzahl der Kurse Anpassungen durchgefuehrt werden
        f = satisfaction - 10*numUn

        # keep satisfaction counts
        if detail:
            if studPref >=1: 
                self.numFirst = numFirst
            if studPref >=2: 
                self.numSecond = numSecond
            if studPref >=3: 
                self.numThird = numThird
            if studPref >=4: 
                self.numFourth = numFourth
            if studPref >=5: 
                self.numFifth = numFifth
            if studPref >=6: 
                self.numSixth = numSixth
            if studPref >=7: 
                self.numSeventh = numSeventh
            if studPref >=8: 
                self.numEighth = numEigth
            if studPref >=9: 
                self.numNinth = numNinth
            if studPref ==10: 
                self.numTenth = numTenth

            # Hier muessen je nach Anzahl der Kurse Anpassungen durchgefuehrt werden
                                               
        # Fitness Rueckgabe
        return f
    # # # # # # # # # # # # # # # # #
    # Ende getFitness(chromosome)   #
    # # # # # # # # # # # # # # # # #


    def choosePop(self):
        # #######################################################
        # Rueckgabe einer population N, with |N| = POP,         #
        # durch Auswahl der besten POP Chromosomen nach Fitness #
        # #######################################################
        """Rueckgabe einer auser der Pre-Population ausgewaehlten Population."""

        # Diktionaer erstellen
        pop = dict()

        # Sortierung der Fitness
        keys = self.pdna.keys()
        for i in range(len(keys)):
            keys[i] = int(keys[i])
        keys.sort()
        keys.reverse()

        # Population substantiieren
        count = 0
        while count < self.POP:
            # naechstes Chromosom finden
            cnt = count
            for i in keys:
                if cnt >= len(self.pdna[str(i)]):
                    cnt -= len(self.pdna[str(i)])
                else:
                    key = str(i)
                    break

            # naechstes Chromosom hinzufuegen
            chro = self.pdna[key][cnt]
            if str(i) in pop.keys():
                pop[str(i)].append(chro)
            else:
                pop[str(i)] = [chro]

            count += 1

        # Rueckgabe der Population
        return pop
    # # # # # # # # # # # #
    # Ende   choosePop()  #
    # # # # # # # # # # # #


    def getMaxFitChros(self):
        # ################################################################
        # Rueckgabe der maximalen Fitness des Chromosoms in dna         #
        #und aller Chromosomen mit diesner Fitness in dna                #
        # ############################################################## #
        """Rueckgabe der maximalen Fitness des Chromosoms in dna."""

        # Maximale Fitness identifizieren
        keys = self.dna.keys()
        for i in range(len(keys)):
            keys[i] = int(keys[i])
        fx = max(keys)

        # Alle Chromosomen mit dieser Fitness in Liste setzen
        fxs = list()
        for i in self.dna[str(fx)]:
            fxs.append(i)

        # Liste zurueckgeben
        return fx, fxs
    # # # # # # # # # # # # # #
    # Ende getMaxFitChros()   #
    # # # # # # # # # # # # # #$


    def getFRatios(self):
        # #######################################################
        # Liste mit Endpunkten fuer die Fitness Ratio Berechnung#
        # #######################################################
        """Liste mit Endpunkten fuer die Fitness Ratio Berechnung."""

        # Ratio-Liste erstellen
        ratios = list()

        # Schluessel und Fitness
        keys = self.dna.keys()

        # Endpunkte in Liste setzen
        prev = 0
        for i in keys:
            curr = (int(i) * len(self.dna[i])) + prev
            ratios.append(curr)
            prev = curr

        # Liste zurueckgeben
        return ratios
    
    # # # # # # # # # # # #
    # Ende getFRatios()   #
    # # # # # # # # # # # #


    def generate(self,fRatios):
        # ######################################### #
        # Chromosomenwahl aus der Population,       #
        # crossover Durchfuehrung,                  #
        # Chromosom zu pdna hinzufuegen             #
        # Rueckgabe der Chromosomenzahl            #
        # # # # # # # # # # # # # # # # # # # # # # #
        # fRatios - Ratioliste                      #
        # ######################################### #
        """Chromosomenwahl, Crossover und Hinzufuegen zu  pdna."""

        # Schluessel
        keys = self.dna.keys()

        # Chromosomenwahl
        #   * hoehere Fitness bedeutet hoehere Auswahlwahrscheinlichkeit
        p = random.randint(0,fRatios[-1])
        curr = 0
        while p > fRatios[curr]:
            curr = curr+1
        
        chros = self.dna[keys[curr]]
        ci = random.randint(0,len(chros)-1)
        c1 = chros[ci]
            
        # Crossoverentscheidung festlegen
        p = random.random()
        if p < self.C: # crossover, fitness generieren und zu Population hinzufuegen
            co1 = self.crossover(c1)
            co1f = self.getFitness(co1, self.studPref)
            if str(co1f) in self.pdna.keys():
                self.pdna[str(co1f)].append(co1)
            else:
                self.pdna[str(co1f)] = [co1]
            return 1  # Rueckgabe der zur Population hinzugefuegten Nummer

        else:
            return 0  # Rueckgabe der zur Population hinzugefuegten Nummer
   
    # # # # # # # # # # # # # # #
    # Ende generate(fRatios)    #
    # # # # # # # # # # # # # # #


    def crossover(self,c):
        # ################################### #
        # Cross-Over eines Chromosoms.        #
        # # # # # # # # # # # # # # # # # # # #
        # c - Chromosom fuer Cross-Over       #
        # ################################### #
        """Cross-Over eines Chromosoms."""

        # zwei Punkte
        b1 = random.randint(0,len(c)-1)
        b2 = b1
        while b2 == b1:
            b2 = random.randint(0,len(c)-1)
        if b2 < b1:
            b1, b2 = b2, b1

        # Teile des Chromosoms
        p1 = c[:b1]
        p2 = c[b1:b2+1]
        p3 = c[b2+1:]

        # Mischung der Mitte
        random.shuffle(p2)

        # Wiedervereinigung
        co = p1 + p2 + p3

        # Rueckgabe des Chromosoms
        return co
    
    # # # # # # # # # # # #
    # Ende crossover(c)   #
    # # # # # # # # # # # #
    

    def improve(self,chromosome):
        # ######################################################################
        # Veraendere Position eines nicht zugewiesenen Studenten s im Chromosom#
        # mit der Position eines Studenten der die erste Praeferenz            #
        # gelistet von s bekommen hat                                          #
        # ######################################################################
        """Returns an chromosome with an order helping an unassigned student."""

        # getFitness aufrufen um  self.classes Diktionaer zu berechnen
        ans = chromosome[:]
        
        # nicht zugewiesenen Studierenden auswaehlen
        unList = self.classes["unassigned"]
        #print unList

        if len(unList)>1:
            student = unList.pop(random.randint(0,len(unList)-1))
        elif len(unList)==1:
            student = unList.pop(0)
        else:
            return ans
        #print student
        wanted = self.infoList[student][1]  #Erste Wahl des gewaehlten Studierenden
        
        # jemanden finden der diesen Kurs hat
        victimList = self.classes[str(wanted)]
        #print victimList
        
        rc=victimList[random.randint(0,len(victimList)-1)]
        #print rc
        #print ans
        
        ind1 = ans.index(student-1)
        ind2 = ans.index(rc-1)
      
        ans[ind1] = rc-1 
        ans[ind2] = student-1
        
        return ans
    # # # # # # # # # # # # # # # #
    # Ende   improve(chromosome)  #
    # # # # # # # # # # # # # # # #


    def mutate(self):
        # ########################################################## #
        # Mutiert M Prozent der bits in der generierten Population.  #
        # ########################################################## #
        """Mutiert die generierte Population."""

        # Anzahl der Mutationen
        mutate = int(self.NUMSTD * self.POP * self.M)

        # Schluessel
        keys = self.pdna.keys()

        # Mutation
        for i in range(mutate):
            # Zufaellige Fitness waehlen
            rf = keys[random.randint(0,len(keys)-1)]

            # zufaelliges Chromosom in dieser Fitness waehlen
            rc = self.pdna[rf][random.randint(0,len(self.pdna[rf])-1)]
            ro = rc[:]

            # Auswahl zweier Austauschstudenten
            r1 = random.randint(0,len(rc)-1)
            r2 = random.randint(0,len(rc)-1)
            while r1 == r2:
                r2 = random.randint(0,len(rc)-1)

            # Werte
            r1v = rc[r1]
            r2v = rc[r2]

            # Austausch
            ro[r1] = r2v
            ro[r2] = r1v

            # altes Chromosom entfernen
            self.pdna[rf].remove(rc)

            # mutiertes Chromosom hinzufuegen
            self.pdna[rf].append(ro)
   
    # # # # # # # # # #
    # Ende   mutate() #
    # # # # # # # # # #


    def merge(self):
        # #################################
        # Fuegt alle Chromosomen aus pdna #    
        # in dna ein und verbessert sie   #
        # #################################
        """ Fuegt alle Chromosomen aus pdna in dna ein und verbessert sie."""

        # Iterationsschluessel
        keys = self.pdna.keys()

        # Chromosomen hinzufuegen
        for i in keys:
            while len(self.pdna[i]) > 0:
                c = self.pdna[i].pop(0)

                # Fitness
                f = self.getFitness(c, self.studPref)

                # Verbesserung
                c = self.improve(c)

                # neue Fitness
                f = self.getFitness(c, self.studPref)

                # Zur dna hinzufuegen
                if str(f) in self.dna.keys():
                    self.dna[str(f)].append(c)
                else:
                    self.dna[str(f)] = [c]
    # # # # # # # # # #
    # Ende   merge()  #
    # # # # # # # # # #


# ############################### #
#       Non-Genetic Methods       #
# # # # # # # # # # # # # # # # # #
# Methoden fuer die Implementierung#
# welche nicht Teil des           #
# Algorithmus sind                #
# ############################# ###

    def getInfoLists(self,infoFile):
        # ######################################################### #
        # Rueckgabe einer Liste mit Kurs und Praeferenzinformationen#
        # undd dem Klassendiktionaer -> classIndex format           #
        # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
        # infoFile - mit der Formatierung                           #
        #       class1 class2 class3 class4  etc                    #
        #       student1 choice1 choice2 choice3 choice4 etc        #
        #       student2 choice1 choice2 choice3 choice4 etc        #
        #       etc.                                                #
        # ######################################################### #
        """Rueckgabe der Kurliste."""

        # Datei einlesen
        lines = infoFile.readlines()

        # Variable erstellen
        rtn = list()

        # Formatierung    
        for i in range(len(lines)):
            info = lines[i].split()
            if i > 0:
                for j in range(1,len(info)):
                    info[j] = int(info[j])
            else:
                for j in range(len(info)):
                    info[j] = int(info[j])
            rtn.append(info)

        # Rueckgabe der Informationsliste
        return rtn
    # # # # # # # # # # # # # # # # #
    # Ende   getInfoLists(infoFile) #
    # # # # # # # # # # # # # # # # #


    def createOutput(self,fxs,dataFile):
        # ####################################################### #
        # Output-Dateien erstellen.                               #
        # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
        # fxs - Liste der Chromosomen mit maximaler Fitness       #
        # ####################################################### #
        """Output-Dateien erstellen."""

        # Chromsom mit geringster Anzahl nicht zugewiesener Studierenden finden
        best = self.NUMSTD
        for i in fxs:
            f = self.getFitness(i, self.studPref)
            if len(self.classes["unassigned"]) < best:
                best = len(self.classes["unassigned"])
                which = i

        # Zuweisungen fuer die besten
        f = self.getFitness(which, self.studPref, True)

            #############################
            ##### Einteilungs-Datei #####
        try:
            schedule = open("schedule.txt",'w')
        except IOError:
            print "Einteilungs-Datei konnte nicht erstellt werden."
            sys.exit(1)

        try:
            dataf = open(dataFile,'r')
            datafconts = dataf.readlines()
            data = range(len(datafconts))
            for i in range(len(datafconts)):
                data[i] = datafconts[i].split()
            dataf.close()
        except IOError:
            print "Rohdaten konnten nicht geladen werden."
            sys.exit(1)

        # Liste mit Plazierungen
        placements = range(self.NUMSTD)
        for i in self.classes:
            for j in self.classes[i]:
                placements[j-1] = i

        # Erstellung der zu schreibenden Informationen
        for i in range(self.NUMSTD):
            ltw = str(data[i][0]) + "\t"
            for j in data[i][1:-self.studPref]:
                # Hier muessen je nach Anzahl der Kurse Anpassungen durchgefuehrt werden
                ltw += str(j) + " "
            ltw += "\t"
            if placements[i] != "unassigned":
                ltw += "\t" + str(data[i][-self.studPref:].index(placements[i])+1) + "\t"
                  # Hier muessen je nach Anzahl der Kurse Anpassungen durchgefuehrt werden
            ltw += str(placements[i]) + "\n"
            schedule.write(ltw)

        # Datei schliessen
        schedule.close
            ##### Einteilungs-Datei #####
            #############################

            #########################
            #####   Kursgroesse  #####
        try:
            size = open("size.txt",'w')
        except IOError:
            print "Datei fuer Kursgroesse konnte nicht erstellt werden."
            sys.exit(1)

        for i, j in self.classes.iteritems():
            writeMe = "Kurs " + str(i) + " hat " + str(len(j)) + " Studierende.\n"
            size.write(writeMe)

        writeMe = "\nAnzahl der ersten Praeferenzen: " + str(self.numFirst) + "\n"
        writeMe += "Anzahl der zweiten Praeferenzen: " + str(self.numSecond) + "\n"
        if self.studPref >=3:
            writeMe += "Anzahl der dritten Praeferenzen: " + str(self.numThird) + "\n"
        if self.studPref >=4:
            writeMe += "Anzahl der vierten Praeferenzen: " + str(self.numFourth) + "\n"
        if self.studPref >=5:
            writeMe += "Anzahl der fuenften Praeferenzen: " + str(self.numFifth) + "\n"
        if self.studPref >=6:
            writeMe += "Anzahl der sechsten Praeferenzen: " + str(self.numSixth) + "\n"
        if self.studPref >=7:
            writeMe += "Anzahl der siebten Praeferenzen: " + str(self.numSeventh) + "\n"
        if self.studPref >=8:
            writeMe += "Anzahl der achten Praeferenzen: " + str(self.numEigth) + "\n"
        if self.studPref >=9:
            writeMe += "Anzahl der neunten Praeferenzen: " + str(self.numNinth) + "\n"
        if self.studPref ==10:
            writeMe += "Anzahl der zehnten Praeferenzen: " + str(self.numTenth) + "\n"
            
            
            
       # writeMe += "Anzahl der dritten Praeferenzen: " + str(self.numFourth) + "\n"
       # Hier muessen je nach Anzahl der Kurse Anpassungen durchgefuehrt werden
        writeMe += "\nFitness: " + str(f) + "\n"
        size.write(writeMe)

        size.close()
     
    # # # # # # # # # # # # # # # # # # #
    # Ende createOutput(fxs,dataFile) #
    # # # # # # # # # # # # # # # # # # #

### Hauptprogramm

print """   Kurszuordnung
Bitte geben Sie die Nummer des Befehls ein, den Sie ausfuehren moechten.
1. Programm starten
2. Programm abbrechen
"""

n = raw_input(": ")

while not n in ['1','2']:
    print "\t", n, "ist keine valide Angabe."
    print
    n = raw_input(": ")

print
if n == '1':
    print "Bitte geben Sie die folgenden Informationen ein:"
    
    print "\tWie viele Studierende werden eingeteilt?"
    numStd = int(raw_input(": "))
    
    print
    print "\tWie viele Studierende duerfen maximal an einem Kurs teilnehmen?"
    maxCls = int(raw_input(": "))
    
    print
    print "\tWelche Datei beinhaltet die Informationen zu den Studierenden?"
    dataFile = raw_input(": ")
    
    print
    print "\tWie viele Praeferenzen sind pro Student in der Datei enthalten (Wertebereich 2 bis 10)?"
    studPref = int(raw_input(": "))


    print
    print "Erstellen der notwendigen Dateien..."
    switch(dataFile,maxCls,studPref)
    print "Die Dateien wurden erstellt. Implementierung des Algorithmus..."
    time.sleep(1)

    print
    o = Ordering(numStd,maxCls,0,dataFile,studPref)
    print "Implementierung erfolgreich abgeschlossen."

    print
    print "In Ihrem Arbeitsverzeichnis wurden folgende Dateien abgespeichert:"
    print "schedule.txt\tbeinhaltet erste Praeferenz und Zuteilung der Studierenden."
    print "size.txt\tbeinhaltet die Anzahl der Studierenden in jedem Kurs."

    print
    print "Schliessen..."
elif n == '2':
    print "Schliessen..."
else:
    print "FEHLER: Unakzeptabler Input!"
    sys.exit(1)

   Kurszuordnung
Bitte geben Sie die Nummer des Befehls ein, den Sie ausfuehren moechten.
1. Programm starten
2. Programm abbrechen

: 1

Bitte geben Sie die folgenden Informationen ein:
	Wie viele Studierende werden eingeteilt?
: 78

	Wie viele Studierende duerfen maximal an einem Kurs teilnehmen?
: 15

	Welche Datei beinhaltet die Informationen zu den Studierenden?
: uebung.txt

	Wie viele Praeferenzen sind pro Student in der Datei enthalten (Wertebereich 2 bis 10)?
: 6

Erstellen der notwendigen Dateien...
{'11': ['1734300', '1748182', '1753695', '1749998', '1748140', '1745527', '1652129', '1690667', '1736025', '1735807', '1750495', '1749940', '1746468', '1735676', '1745155', '1725956', '1739204', '1725165', '1746756', '1751436', '1724952', '1750411', '1749867', '1738153', '1735498', '1735105', '1744172', '1738347', '1735194', '1727780', '1746185', '1747079', '1750956', '1704522', '1726169', '1742458', '1749406', '1746960', '1749212', '1735372', '1687738', '1749249', '1728103', '174